# Tool use and agents

用 Langchain 官网的例子来实现 Agent 和 Tools 的使用

[参考资料](https://python.langchain.com/docs/use_cases/tool_use/quickstart/)

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
import getpass
import os

os.environ["OPENAI_API_KEY"] = "{your_key}"
# os.environ["OPENAI_API_KEY"] = getpass.getpass()


# llm = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     temperature=0
# )

llm = ChatOpenAI(
    model_name="qwen32b-distill-r1",
    openai_api_key="EMPTY",
    openai_api_base="http://10.4.119.108:8000/v1",
    # http_client=http_client
)

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


In [3]:
get_word_length.invoke("abc")

3

创建工具

In [4]:
from langchain_core.tools import tool

# 利用 tool 装饰器将函数注册为工具
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

# 选择多个工具
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


In [5]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [6]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

llm_with_tools = llm.bind_tools([multiply, add, exponentiate])

msg = llm_with_tools.invoke("whats 5 powers 4, and then add 3, and then add 6?")

# 官方文档的例子，但是不知道为什么 msg 中不包含 tool_calls
# 原因是 AIMessage 中并没有 tool_calls 这个属性，而是 additional_kwargs 中的 tool_calls
# 且 additional_kwargs 是一个字典，所以需要使用字典的方式来访问，后续调用要转化成字典的方式

msg.tool_calls
# msg.additional_kwargs['tool_calls'][0]["function"]["arguments"]

APIConnectionError: Connection error.

In [19]:
from operator import itemgetter
import json

# 这里第二步的 Lambda 函数是为了将 tool_calls 中的字符串转化为字典

# 这里的源码重载了 | 运算符，使得可以将多个函数串联起来，也就是 __or__ 方法
# langchain_core/runnables/base.py
# def __or__(
#     self,
#     other: Union[
#         Runnable[Any, Other],
#         Callable[[Any], Other],
#         Callable[[Iterator[Any]], Iterator[Other]],
#         Mapping[str, Union[Runnable[Any, Other], Callable[[Any], Other], Any]],
#     ],
# ) -> RunnableSerializable[Input, Other]:
#     """Compose this runnable with another object to create a RunnableSequence."""
#     return RunnableSequence(self, coerce_to_runnable(other))

# chain = llm_with_tools | (lambda x: json.loads(x.additional_kwargs['tool_calls'][0]["function"]["arguments"])) | multiply
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 23")

92

## Using Agent

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

ImportError: Could not import langchainhub, please install with `pip install langchainhub`.

In [10]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [11]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
question = "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
result = agent_executor.invoke(
    {
        "input": question
    }
)
result



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 405, 'exponent': 2}`


164025The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three is 3645. 

When you square this result, you get 164025.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three is 3645. \n\nWhen you square this result, you get 164025.'}

In [14]:
from langchain_core.messages import HumanMessage

chat_history = []
chat_history.extend([HumanMessage(content=result["input"]), result["output"]])
question_new = "Then double the value?"
new_result = agent_executor.invoke({"input": question_new, "chat_history": chat_history})
new_result



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'first_int': 13286025, 'second_int': 2}`


26572050Doubling the value of 13,286,025 gives 26,572,050.

> Finished chain.


{'input': 'Then double the value?',
 'chat_history': [HumanMessage(content='Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result'),
  'The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.'],
 'output': 'Doubling the value of 13,286,025 gives 26,572,050.'}